### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [3]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [5]:
review_agg1 = reviews.groupby('movie_id').agg({'rating':'mean'}).reset_index().rename(columns = {'rating':'mean_rating'})
review_agg2 = reviews.groupby('movie_id').agg({'rating':'count'}).reset_index().rename(columns ={'rating':'count_rating'})
most_recent = reviews.groupby('movie_id').agg({'timestamp':'max'}).reset_index()
review_latest = pd.merge(most_recent, reviews[['movie_id','timestamp','rating']], \
                         how='inner', on =['movie_id','timestamp']).rename(columns={'rating':'latest_rating'}).\
                            drop(columns='timestamp')
merged = pd.merge(review_agg1, review_agg2, how = 'inner', on = 'movie_id')
merged = merged.loc[merged['count_rating']>=5,]
merged2 = pd.merge(merged, review_latest, how = 'left', on = 'movie_id').\
            sort_values(by=['mean_rating','count_rating','latest_rating'], ascending=[False, False, False])
top_movies = pd.merge(merged2.head(10)[['movie_id']], movies, how = 'left', on='movie_id')['movie'].tolist()
top_movies

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 'Crawl Bitch Crawl (2012)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Romeo Juliet (2009)',
 'Third Contact (2011)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)']

In [57]:
merged2.head(30)

,movie_id,mean_rating,count_rating,latest_rating
9432,4921860,10.000000,48,10
9584,5262972,10.000000,28,10
9718,5688932,10.000000,14,10
8035,2737018,10.000000,10,10
7279,2219210,10.000000,6,10
7883,2560840,10.000000,6,10
5652,1431149,10.000000,5,10
6958,2059318,10.000000,5,10
9238,4448444,10.000000,5,10
9532,5131914,10.000000,5,10


#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [7]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    review_agg1 = reviews.groupby('movie_id').agg({'rating':'mean'}).reset_index().\
                    rename(columns = {'rating':'mean_rating'})
    review_agg2 = reviews.groupby('movie_id').agg({'rating':'count'}).reset_index().\
                    rename(columns ={'rating':'count_rating'})
    most_recent = reviews.groupby('movie_id').agg({'timestamp':'max'}).reset_index()
    review_latest = pd.merge(most_recent, reviews[['movie_id','timestamp','rating']], \
                         how='inner', on =['movie_id','timestamp']).rename(columns={'rating':'latest_rating'}).\
                            drop(columns='timestamp')
    merged = pd.merge(review_agg1, review_agg2, how = 'inner', on = 'movie_id')
    merged = merged.loc[merged['count_rating']>=5,]
    merged2 = pd.merge(merged, review_latest, how = 'left', on = 'movie_id').\
            sort_values(by=['mean_rating','count_rating','latest_rating'], ascending=[False, False, False])
    top_movies = pd.merge(merged2.head(n_top)[['movie_id']], movies, how = 'left', on='movie_id')['movie'].tolist()
    
    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [9]:
popular_recommendations(70000, 20)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 'Crawl Bitch Crawl (2012)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Romeo Juliet (2009)',
 'Third Contact (2011)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'Agnelli (2017)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']

In [53]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = ['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 'Crawl Bitch Crawl (2012)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Romeo Juliet (2009)',
 'Third Contact (2011)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'Agnelli (2017)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)'] # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = ['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 'Crawl Bitch Crawl (2012)']# Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = ['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 'Crawl Bitch Crawl (2012)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Romeo Juliet (2009)',
 'Third Contact (2011)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'Agnelli (2017)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)',
 'Tengoku to jigoku (1963)',
 'Kirik Party (2016)',
 'Beyond the Sea (2004)',
 'Chasing Asylum (2016)',
 'Blood Brother (2013)',
 'Poshter Girl (2016)',
 'The Adventures of Robin Hood (1938)',
 'Umberto D. (1952)',
 'Bridegroom (2013)',
 'Nashville (1975)',
 'Mad As Hell (2014)',
 'No Stone Unturned (2017)',
 'The Shawshank Redemption (1994)',
 'A Matter of Life and Death (1946)',
 'Presto (2008)',
 'Everything or Nothing: The Untold Story of 007 (2012)',
 'Cyborgs: Heroes Never Die (2017)',
 'Dag II (2016)',
 'The Lord of the Rings: The Return of the King (2003)',
 'Sanshô dayû (1954)',
 "Hillary's America: The Secret History of the Democratic Party (2016)",
 'Seppuku (1962)',
 'Chasing Coral (2017)',
 'The Phantom of the Opera at the Royal Albert Hall (2011)',
 'Terra (2015)',
 'Hidden Colors (2011)',
 'Berserk: Ohgon jidai hen 3 - Kôrin (2013)',
 'Men at Lunch (2012)',
 'The Dark Knight (2008)',
 'Andaz Apna Apna (1994)',
 'Przypadek (1987)',
 "La passion de Jeanne d'Arc (1928)",
 'Jalsaghar (1958)',
 'Ayla: The Daughter of War (2017)',
 'The Decline of Western Civilization (1981)',
 'Lakshya (2004)',
 "Bab'Aziz (2005)",
 'Kites (2010)',
 'Who is Dayani Cristal? (2013)',
 'A Face in the Crowd (1957)',
 'Southern Comfort (1981)',
 'Notre musique (2004)',
 'Zinda Bhaag (2013)',
 '12 Angry Men (1957)',
 'Taare Zameen Par (2007)',
 'Night Will Fall (2014)',
 'Nema-ye Nazdik (1990)',
 'Advise & Consent (1962)',
 'Hitting the Apex (2015)',
 'To Be or Not to Be (1942)',
 'Le notti di Cabiria (1957)',
 'Ma nuit chez Maud (1969)',
 'Colombia magia salvaje (2015)',
 'The Message (1977)',
 'Moskva slezam ne verit (1979)',
 'Shelter (2007)',
 'Aparajito (1956)',
 'Apur Sansar (1959)',
 'The Godfather: Part II (1974)',
 'Amadeus (1984)',
 'Terminator 2: Judgment Day (1991)',
 'Letyat zhuravli (1957)',
 'Messi (2014)',
 "It's a Wonderful Life (1946)",
 'I Believe in Miracles (2015)',
 'Dai-bosatsu tôge (1966)',
 'Umut (1970)',
 'Le trou (1960)',
 'Cowboy Bebop: Tengoku no tobira (2001)',
 'Roger Waters the Wall (2014)',
 'Ordet (1955)',
 'Wild Wild Country (2018)',
 'Il buono, il brutto, il cattivo. (1966)',
 'Forrest Gump (1994)',
 'The Green Mile (1999)',
 "Schindler's List (1993)",
 'The Godfather (1972)',
 'Bajrangi Bhaijaan (2015)',
 'Ikiru (1952)',
 "One Flew Over the Cuckoo's Nest (1975)"]# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = ['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 'Crawl Bitch Crawl (2012)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Romeo Juliet (2009)',
 'Third Contact (2011)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'Agnelli (2017)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)',
 'Tengoku to jigoku (1963)',
 'Kirik Party (2016)',
 'Beyond the Sea (2004)',
 'Chasing Asylum (2016)',
 'Blood Brother (2013)',
 'Poshter Girl (2016)',
 'The Adventures of Robin Hood (1938)',
 'Umberto D. (1952)',
 'Bridegroom (2013)',
 'Nashville (1975)',
 'Mad As Hell (2014)',
 'No Stone Unturned (2017)',
 'The Shawshank Redemption (1994)',
 'A Matter of Life and Death (1946)',
 'Presto (2008)']# Your solution list here



In [54]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

AssertionError: The first check failed...

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```